In [1]:
import numpy as np
import pandas as pd
import os
import time
os.getcwd()
os.chdir('C:\\Users\\Wetauzer\\Desktop\\Data')

In [2]:
tx_df = pd.read_csv('Z:03_22_03_26.csv', nrows = 1000000)
tx_df = tx_df.sort_values('block_number')
tx_df.reset_index(inplace=True)
tx_df = tx_df.loc[:, tx_df.columns.intersection(['block_number','hash', 'gas_price'])]
tx_df = tx_df.rename(columns = {'block_number':'block','hash':'hash', 'gas_price':'gas_price'})
tx_df.gas_price = tx_df.gas_price/1e9 # convert the prices to gwei
tx_df = tx_df[tx_df['gas_price']!=0] # Getting rid of zero gas price transactions
block_df = pd.DataFrame(tx_df["block"].copy().unique()).rename(columns = {0: 'block'})

Below is the function for the "price oracle". It returns a dataframe of either all the transactions or all the blocks with quantile data from the last N blocks (return_tx argument determines if it will be blocks or transactions). By default it returns the min/max and 50th, 75th, and 90th percentiles since those are what oracles seem to use but you can change that. Window tells it how many of the previous blocks to look at. By default, N = 10 blocks, so the output will be a dataframe starting with transactions from the Nth block with summary statistics from the first N blocks.

In [3]:
def oracle_prices(tx_df, block_df, return_tx = True, low = 50, medium = 75, high = 90, window = 10):
    """Gives min/max and 3 quantile values over the last N blocks"""
    blx_in = block_df['block'] # blocks from our dataset
    blx_out = [] # blocks that will have oracle values
    p1 = []
    p2 = []
    p3 = []
    p4 = []
    p5 = []
    for blk in blx_in:
        if blk >= blx_in[window - 1]:
            idx = pd.Index(list(blx_in)).get_loc(blk)
            blk_win = blx_in[idx - window + 1: idx + 1]
            blx_out.append(blk)
            p1.append(tx_df[tx_df.block.isin(blk_win)].gas_price.min())
            p2.append(tx_df[tx_df.block.isin(blk_win)].gas_price.quantile(low/100))
            p3.append(tx_df[tx_df.block.isin(blk_win)].gas_price.quantile(medium/100))
            p4.append(tx_df[tx_df.block.isin(blk_win)].gas_price.quantile(high/100))
            p5.append(tx_df[tx_df.block.isin(blk_win)].gas_price.max())
    blx_oracle_dict = {'block':blx_out, 'minimum':p1, 'low':p2, 'average':p3, 'fast':p4, 'maximum':p5}
    blk_oracle_df = pd.DataFrame(blx_oracle_dict)
    tx_oracle_df = pd.merge(tx_df, blk_oracle_df, how="right", on=["block"])
    if return_tx:
        return tx_oracle_df
    else:
        return blk_oracle_df

In [4]:
oracle_prices(tx_df, block_df, window = 20)

,hash,block,gas_price,minimum,low,average,fast,maximum
0,0xdbfca431021a6483453d8b354adcab97ceb68bb97fdf...,12104904,205.000000,1.0,190.0,204.00,216.0,239.000000
1,0x19f3548849d25666b58889e91a981a9fe8df8ebe97ab...,12104929,215.000000,1.0,190.0,205.00,216.0,239.000000
2,0x2998f228b7a39813aef7c81f0d36dd653c7a57e10ed2...,12104935,173.000000,1.0,189.0,205.00,216.0,239.000000
3,0x71824c00cbe577b9059275622cedb57b70317ef9d666...,12104940,250.000000,1.0,189.0,205.35,220.0,250.000000
4,0xd5404100056b159734bd7da17b5925ba4a4ffcb67f31...,12104951,175.000000,1.0,189.0,205.35,220.0,250.000000
...,...,...,...,...,...,...,...,...
999249,0xd99732d6eaac32aeeea6d4b2a72d2320fc0e12ae9e50...,12116291,166.000000,1.0,171.0,202.00,218.0,4943.685021
999250,0xff50bd1ac596a9838fd26df899062f54775481bc5dae...,12116291,137.000000,1.0,171.0,202.00,218.0,4943.685021
999251,0x64957738c1d9ea9bc99d19243004f93dfe0975d503f3...,12116291,135.000001,1.0,171.0,202.00,218.0,4943.685021
999252,0x92846fdfd02b01f6d65e90097b15c349de82a0300113...,12116291,135.000001,1.0,171.0,202.00,218.0,4943.685021


In [5]:
oracle_prices(tx_df, block_df, return_tx=False, window = 20)

,block,minimum,low,average,fast,maximum
0,12104904,1.0,190.0,204.00,216.00000,239.000000
1,12104929,1.0,190.0,205.00,216.00000,239.000000
2,12104935,1.0,189.0,205.00,216.00000,239.000000
3,12104940,1.0,189.0,205.35,220.00000,250.000000
4,12104951,1.0,189.0,205.35,220.00000,250.000000
...,...,...,...,...,...,...
5645,12116287,1.0,178.0,202.00,218.21250,4943.685021
5646,12116288,1.0,174.0,202.00,218.21250,4943.685021
5647,12116289,1.0,175.0,202.00,218.10625,4943.685021
5648,12116290,1.0,170.0,202.00,218.00000,4943.685021
